# Inspect and summarize SLURM benchmarks

## Overview

## Load, merge and compare benchmark summary `csv`s

We need a function to compare our SLURM summary output files and highlight differences in a table.

In [1]:
library(dplyr)
library(tidyr)

read_and_summarize_differences <- function(directory) {
  # List all CSV files in the directory
  files <- list.files(directory, pattern = "\\.csv$", full.names = TRUE)
  
  # Read each file and store in a list
  data_list <- lapply(files, function(file) {
    data <- read.csv(file)
    # Add a column to identify the source file
    data$Source <- basename(file)
    return(data)
  })
  
  # Combine all data frames into one
  combined_data <- bind_rows(data_list)
  
  # Spread the data to wide format for comparison
  wide_data <- spread(combined_data, key = Parameter, value = Value)
  
  # Identify rows with at least one difference in values
  differing_rows <- wide_data %>%
    summarise(across(-Source, ~n_distinct(.) > 1)) %>%
    select(which(colSums(.) > 0)) %>%
    names()
  
  # Filter the combined data to include only differing parameters
  summary_data <- combined_data %>%
    filter(Parameter %in% differing_rows) %>%
    select(Parameter, Source, Value)
  
  # Pivot wider for a clearer summary table, if necessary
  summary_table <- summary_data %>%
    pivot_wider(names_from = Source, values_from = Value)
  
  return(summary_table)
}


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [2]:
directory <- "/dcs04/lieber/statsgen/mnagle/mwas/CpGWAS/output"
summary_table <- read_and_summarize_differences(directory)
summary_table

Parameter,libd_chr1-chr1_AA-libd_chr1-chr1_AA-1000000-1001000-dynamic-1corestotal-1corepera-20240131-122519-summary.csv,libd_chr1-chr1_AA-libd_chr1-chr1_AA-1000000-1001000-dynamic-1corestotal-allcorepera-20240131-122521-summary.csv,libd_chr1-chr1_AA-libd_chr1-chr1_AA-1000000-1001000-dynamic-8corestotal-allcorepera-20240131-122522-summary.csv,libd_chr1-chr1_AA-libd_chr1-chr1_AA-1000000-1001000-static-1corestotal-1corepera-20240131-122515-summary.csv,libd_chr1-chr1_AA-libd_chr1-chr1_AA-1000000-1001000-static-1corestotal-allcorepera-20240131-122517-summary.csv,libd_chr1-chr1_AA-libd_chr1-chr1_AA-1000000-1001000-static-8corestotal-allcorepera-20240131-122518-summary.csv
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
cores_per_alpha,1,all,all,1,all,all
num_cores,1,1,8,1,1,8
tag,libd_chr1-chr1_AA-1000000-1001000-dynamic-1corestotal-1corepera-20240131-122519,libd_chr1-chr1_AA-1000000-1001000-dynamic-1corestotal-allcorepera-20240131-122521,libd_chr1-chr1_AA-1000000-1001000-dynamic-8corestotal-allcorepera-20240131-122522,libd_chr1-chr1_AA-1000000-1001000-static-1corestotal-1corepera-20240131-122515,libd_chr1-chr1_AA-1000000-1001000-static-1corestotal-allcorepera-20240131-122517,libd_chr1-chr1_AA-1000000-1001000-static-8corestotal-allcorepera-20240131-122518
cv_eval_mode,dynamic,dynamic,dynamic,static,static,static
time_started,2024-01-31 12:25:34,2024-01-31 12:25:34,2024-01-31 13:00:31,2024-01-31 12:25:20,2024-01-31 12:25:34,2024-01-31 12:42:26
time_finished,2024-02-01 02:28:56,2024-02-01 05:45:21,2024-01-31 23:25:58,2024-01-31 16:51:32,2024-01-31 18:04:38,2024-01-31 14:47:18
runtime,14:03:22,17:19:46,10:25:26,04:26:11,05:39:04,02:04:51
type_CPU,Intel(R) Xeon(R) CPU E5-2650L v3 @ 1.80GHz,Intel(R) Xeon(R) CPU E5-2650L v3 @ 1.80GHz,AMD EPYC 7713 64-Core Processor,Intel(R) Xeon(R) CPU E5-2650L v3 @ 1.80GHz,Intel(R) Xeon(R) CPU E5-2650L v3 @ 1.80GHz,AMD EPYC 7713 64-Core Processor
amount_RAM,0.122832637280226,0.122735276818275,0.983715858310461,0.122832637280226,0.122832637280226,0.983715858310461


## Compute core-hours per test

In [3]:
windows <- c(1000,2000,5000,10000,20000,50000,100000,500000)
n_windows <- length(windows)
alphas <- seq(0, 1, .25)
n_alphas <- length(alphas)
chunks <- 1001
n_tests <- n_windows * n_alphas * chunks
n_tests

[1] 40040

In [4]:
colnames(summary_table)

[1] "Parameter"                                                                                                      
[2] "libd_chr1-chr1_AA-libd_chr1-chr1_AA-1000000-1001000-dynamic-1corestotal-1corepera-20240131-122519-summary.csv"  
[3] "libd_chr1-chr1_AA-libd_chr1-chr1_AA-1000000-1001000-dynamic-1corestotal-allcorepera-20240131-122521-summary.csv"
[4] "libd_chr1-chr1_AA-libd_chr1-chr1_AA-1000000-1001000-dynamic-8corestotal-allcorepera-20240131-122522-summary.csv"
[5] "libd_chr1-chr1_AA-libd_chr1-chr1_AA-1000000-1001000-static-1corestotal-1corepera-20240131-122515-summary.csv"   
[6] "libd_chr1-chr1_AA-libd_chr1-chr1_AA-1000000-1001000-static-1corestotal-allcorepera-20240131-122517-summary.csv" 
[7] "libd_chr1-chr1_AA-libd_chr1-chr1_AA-1000000-1001000-static-8corestotal-allcorepera-20240131-122518-summary.csv"

In [5]:
summary_table <- as.data.frame(t(summary_table))
colnames(summary_table) <- summary_table[1, ]
summary_table <- summary_table[-1, ]
summary_table$number_cores <- as.numeric(summary_table$number_cores)

In [6]:
summary_table$runtime_seconds <- sapply(summary_table$runtime, function(x) {
  parts <- strsplit(x, ":")[[1]]
  as.numeric(parts[1]) * 3600 + as.numeric(parts[2]) * 60 + as.numeric(parts[3])
})

In [7]:
summary_table$runtime_hours <- summary_table$runtime_seconds/3600

In [8]:
summary_table$runtime_core_hours <- summary_table$runtime_hours * summary_table$number_cores
summary_table$runtime_core_seconds <- summary_table$runtime_seconds * summary_table$number_cores
summary_table$core_hours_per_test <- summary_table$runtime_core_hours / n_tests
summary_table$core_seconds_per_test <- summary_table$runtime_core_seconds / n_tests

In [9]:
summary_table[which.min(summary_table$core_hours_per_test), ]

,cores_per_alpha,num_cores,tag,cv_eval_mode,time_started,time_finished,runtime,type_CPU,amount_RAM,number_cores,physical_cores,scaffold_ID,runtime_seconds,runtime_hours,runtime_core_hours,runtime_core_seconds,core_hours_per_test,core_seconds_per_test
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
libd_chr1-chr1_AA-libd_chr1-chr1_AA-1000000-1001000-static-1corestotal-1corepera-20240131-122515-summary.csv,1,1,libd_chr1-chr1_AA-1000000-1001000-static-1corestotal-1corepera-20240131-122515,static,2024-01-31 12:25:20,2024-01-31 16:51:32,04:26:11,Intel(R) Xeon(R) CPU E5-2650L v3 @ 1.80GHz,0.122832637280226,1,24,libd_chr1-chr1_AA-libd_chr1-chr1_AA-1000000-1001000-static-1corestotal-1corepera-20240131-122515,15971,4.436389,4.436389,15971,0.0001107989,0.3988761


In [10]:
data.table::fwrite(summary_table, "05-OUT_SLURM_benchmarks_a1.csv")

In [11]:
getwd()

[1] "/dcs04/lieber/statsgen/mnagle/mwas/CpGWAS/scripts"

## How long for whole dataset?

In [12]:
n_sites <- 26000000

In [13]:
n_sites*0.0001107989

[1] 2880.771